# Exercise05 : Distributed Training

Here we change our sample (see "[Exercise03 : Just Train in Your Working Machine](https://github.com/tsmatz/azure-ml-tensorflow-complete-sample/blob/master/notebooks/exercise03_train_simple.ipynb)") for distributed training using multiple machines, and run on [Batch AI](https://tsmatz.wordpress.com/2018/01/30/azure-batch-ai-how-it-works/).   
Now we use Horovod framework using built-in ```azureml.train.dnn.TensorFlow``` estimator. (You can customize your distributed training as you like using base ```azureml.train.estimator.Estimator```.)

*back to [index](https://github.com/tsmatz/azure-ml-tensorflow-complete-sample/)*

## Save your training script as file (train.py)

Create ```scirpt``` directory.

In [1]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

Change our original source code ```train.py``` (see "[Exercise03 : Just Train in Your Working Machine](https://github.com/tsmatz/azure-ml-tensorflow-complete-sample/blob/master/notebooks/exercise03_train_simple.ipynb)") as follows. The lines commented "##### modified" is modified lines.    
After that, please add the following ```%%writefile``` at the beginning of the source code and run this cell.    
This source code is saved as ```./script/train_horovod.py```.

In [5]:
%%writefile script/train_horovod.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
import shutil
import argparse
import math

import tensorflow as tf
import horovod.tensorflow as hvd ##### modified

FLAGS = None
batch_size = 100

#
# define functions for Estimator
#

def _my_input_fn(filepath, num_epochs):
    # image - 784 (=28 x 28) elements of grey-scaled integer value [0, 1]
    # label - digit (0, 1, ..., 9)
    data_queue = tf.train.string_input_producer(
        [filepath],
        num_epochs = num_epochs) # data is repeated and it raises OutOfRange when data is over
    data_reader = tf.TFRecordReader()
    _, serialized_exam = data_reader.read(data_queue)
    data_exam = tf.parse_single_example(
        serialized_exam,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })
    data_image = tf.decode_raw(data_exam['image_raw'], tf.uint8)
    data_image.set_shape([784])
    data_image = tf.cast(data_image, tf.float32) * (1. / 255)
    data_label = tf.cast(data_exam['label'], tf.int32)
    data_batch_image, data_batch_label = tf.train.batch(
        [data_image, data_label],
        batch_size=batch_size)
    return {'inputs': data_batch_image}, data_batch_label

def _get_input_fn(filepath, num_epochs):
    return lambda: _my_input_fn(filepath, num_epochs)

def _my_model_fn(features, labels, mode):
    # with tf.device(...): # You can set device if using GPUs

    # define network and inference
    # (simple 2 fully connected hidden layer : 784->128->64->10)
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal(
                [784, FLAGS.first_layer],
                stddev=1.0 / math.sqrt(float(784))),
            name='weights')
        biases = tf.Variable(
            tf.zeros([FLAGS.first_layer]),
            name='biases')
        hidden1 = tf.nn.relu(tf.matmul(features['inputs'], weights) + biases)
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal(
                [FLAGS.first_layer, FLAGS.second_layer],
                stddev=1.0 / math.sqrt(float(FLAGS.first_layer))),
            name='weights')
        biases = tf.Variable(
            tf.zeros([FLAGS.second_layer]),
            name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal(
                [FLAGS.second_layer, 10],
                stddev=1.0 / math.sqrt(float(FLAGS.second_layer))),
        name='weights')
        biases = tf.Variable(
            tf.zeros([10]),
            name='biases')
        logits = tf.matmul(hidden2, weights) + biases
 
    # compute evaluation matrix
    predicted_indices = tf.argmax(input=logits, axis=1)
    if mode != tf.estimator.ModeKeys.PREDICT:
        label_indices = tf.cast(labels, tf.int32)
        accuracy = tf.metrics.accuracy(label_indices, predicted_indices)
        tf.summary.scalar('accuracy', accuracy[1]) # output to TensorBoard
 
        loss = tf.losses.sparse_softmax_cross_entropy(
            labels=labels,
            logits=logits)
 
    # define operations
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_or_create_global_step()        
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=FLAGS.learning_rate)
        optimizer = hvd.DistributedOptimizer(optimizer) ##### modified
        train_op = optimizer.minimize(
            loss=loss,
            global_step=global_step)
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            train_op=train_op)
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            'accuracy': accuracy
        }
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            eval_metric_ops=eval_metric_ops)
    if mode == tf.estimator.ModeKeys.PREDICT:
        probabilities = tf.nn.softmax(logits, name='softmax_tensor')
        predictions = {
            'classes': predicted_indices,
            'probabilities': probabilities
        }
        export_outputs = {
            'prediction': tf.estimator.export.PredictOutput(predictions)
        }
        return tf.estimator.EstimatorSpec(
            mode,
            predictions=predictions,
            export_outputs=export_outputs)

def _my_serving_input_fn():
    inputs = {'inputs': tf.placeholder(tf.float32, [None, 784])}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

#
# Main
#

parser = argparse.ArgumentParser()
parser.add_argument(
    '--data_folder',
    type=str,
    default='./data',
    help='Folder path for input data')
parser.add_argument(
    '--chkpoint_folder',
    type=str,
    default='./logs',  # AML experiments logs folder
    help='Folder path for checkpoint files')
parser.add_argument(
    '--model_folder',
    type=str,
    default='./outputs',  # AML experiments outputs folder
    help='Folder path for model output')
parser.add_argument(
    '--learning_rate',
    type=float,
    default='0.07',
    help='Learning Rate')
parser.add_argument(
    '--first_layer',
    type=int,
    default='128',
    help='Neuron number for the first hidden layer')
parser.add_argument(
    '--second_layer',
    type=int,
    default='64',
    help='Neuron number for the second hidden layer')
FLAGS, unparsed = parser.parse_known_args()

# clean checkpoint and model folder if exists
if os.path.exists(FLAGS.chkpoint_folder) :
    for file_name in os.listdir(FLAGS.chkpoint_folder):
        file_path = os.path.join(FLAGS.chkpoint_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
if os.path.exists(FLAGS.model_folder) :
    for file_name in os.listdir(FLAGS.model_folder):
        file_path = os.path.join(FLAGS.model_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

hvd.init() ##### modified

# read TF_CONFIG
run_config = tf.contrib.learn.RunConfig()

# create Estimator
mnist_fullyconnected_classifier = tf.estimator.Estimator(
    model_fn=_my_model_fn,
    model_dir=FLAGS.chkpoint_folder if hvd.rank() == 0 else None, ##### modified
    config=run_config)
train_spec = tf.estimator.TrainSpec(
    input_fn=_get_input_fn(os.path.join(FLAGS.data_folder, 'train.tfrecords'), 2),
    #max_steps=60000 * 2 / batch_size)
    max_steps=(60000 * 2 / batch_size) // hvd.size(), ##### modified
    hooks=[hvd.BroadcastGlobalVariablesHook(0)]) ##### modified
eval_spec = tf.estimator.EvalSpec(
    input_fn=_get_input_fn(os.path.join(FLAGS.data_folder, 'test.tfrecords'), 1),
    steps=10000 * 1 / batch_size,
    start_delay_secs=0)

# run !
tf.estimator.train_and_evaluate(
    mnist_fullyconnected_classifier,
    train_spec,
    eval_spec
)

# save model and variables
if hvd.rank() == 0 : ##### modified
    model_dir = mnist_fullyconnected_classifier.export_savedmodel(
        export_dir_base = FLAGS.model_folder,
        serving_input_receiver_fn = _my_serving_input_fn)
    print('current working directory is ', os.getcwd())
    print('model is saved ', model_dir)

Overwriting script/train_horovod.py


## Train on multiple machines (Horovod)

### Step 1 : Get workspace setting

Before starting, you must read your configuration settings. (See "[Exercise01 : Prepare Config Settings](https://github.com/tsmatz/azure-ml-tensorflow-complete-sample/blob/master/notebooks/exercise01_prepare_config.ipynb)".)

In [3]:
from azureml.core import Workspace
import azureml.core

ws = Workspace.from_config()

Found the config file in: /data/home/tsmatsuz/Tutorial/aml_config/config.json


### Step 2 : Create multiple virtual machines (cluster)

Create your new computing cluster using [Batch AI](https://tsmatz.wordpress.com/2018/01/30/azure-batch-ai-how-it-works/). Here we enable auto-scaling from 0 to 4, and then you can save money (all nodes are terminated) if it's inactive.    
If already exists, this script will get the existing cluster.

In [4]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='mycluster01')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = BatchAiCompute.provisioning_configuration(
        vm_size='Standard_D2_v2', 
        autoscale_enabled=True,
        cluster_min_nodes=0,
        cluster_max_nodes=4)
    compute_target = ComputeTarget.create(ws, 'mycluster01', compute_config)
    compute_target.wait_for_completion(show_output=True)

creating new.
Creating
succeeded
BatchAI wait for completion finished
Minimum number of nodes requested have been provisioned


In [6]:
# get a status for the current cluster.
print(compute_target.status.serialize())

{'allocationState': 'steady', 'allocationStateTransitionTime': '2018-11-14T04:28:39.174000+00:00', 'creationTime': '2018-11-14T04:28:13.872000+00:00', 'currentNodeCount': 0, 'errors': None, 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'succeeded', 'provisioningStateTransitionTime': '2018-11-14T04:28:38.589000+00:00', 'scaleSettings': {'manual': None, 'autoScale': {'maximumNodeCount': 4, 'minimumNodeCount': 0, 'initialNodeCount': 0}}, 'vmPriority': 'dedicated', 'vmSize': 'STANDARD_D2_V2'}


### Step 3 : Prepare datastore

You can mount your datastore (See "[Exercise02 : Prepare Datastore](https://github.com/tsmatz/azure-ml-tensorflow-complete-sample/blob/master/notebooks/exercise02_prepare_datastore.ipynb)") into your Batch AI compute.

In [7]:
from azureml.core import Datastore

# get your datastore (See "Exercise 02 : Prepare Datastore")
ds = Datastore.get(ws, datastore_name="myblob01")
ds_data = ds.path('tfdata')

### Step 4 : Generate estimator **

Run distributed training by Horovod using built-in ```azureml.train.dnn.TensorFlow``` estimator.    
If you want to customize more detailed settings (other frameworks, custom images, etc), please use base ```azureml.train.estimator.Estimator``` (parent class).

** Note : This estimator (```azureml.train.dnn.TensorFlow```) is an estimator in AML SDK, and not the same as ```tf.estimator.Estimator``` in TensorFlow. Do not confused for the terminology "Estimator".

In [8]:
from azureml.train.dnn import TensorFlow

script_params={
    '--data_folder': ds_data
}
estimator = TensorFlow(
    source_directory='./script',
    compute_target=compute_target,
    script_params=script_params,
    entry_script='train_horovod.py',
    node_count=2,
    process_count_per_node=1,
    distributed_backend='mpi',
    use_gpu=False)

### Step 5 : Run script and wait for completion

In [9]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='tf_distribued')
run = exp.submit(estimator)

In [10]:
run.wait_for_completion(show_output=True)

RunId: matsu_test1115_06_1542170007548

Streaming azureml-logs/60_control_log_rank_0.txt

This is an MPI job. Rank:0
Streaming log file azureml-logs/60_control_log_rank_0.txt
Streaming log file azureml-logs/80_driver_log_rank_0.txt

Streaming azureml-logs/80_driver_log_rank_0.txt

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
2018-11-14 04:43:02.556552: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
current working directory is  /mnt/batch/tasks/shared/LS_root/jobs/mycluster01d83507911b/azureml/matsu_test1115_06_1542170007548/mounts/azureml_project_share/azureml/matsu_test1115_06_1542170007548
model is saved  b'./model/1542170598'


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service

Execution Summary
RunId: matsu_test1115_06_1542170007548



{'runId': 'matsu_test1115_06_1542170007548',
 'target': 'mycluster01',
 'status': 'Finalizing',
 'startTimeUtc': '2018-11-14T04:40:02.097692Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '39e55cbb-0e5c-4906-891c-c2c55c77f3bc'},
 'runDefinition': {'Script': 'train_horovod.py',
  'Arguments': ['--data_folder',
   '$AZUREML_DATAREFERENCE_ff1877de6af641928f73feb60b347483'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 2,
  'Target': 'mycluster01',
  'DataReferences': {'ff1877de6af641928f73feb60b347483': {'DataStoreName': 'myblob01',
    'Mode': 'Mount',
    'PathOnDataStore': 'tfdata',
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 2,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
     

### Step 6 : Remove Batch AI compute

**You don't need to remove your Batch AI** for saving money, because the nodes will be automatically terminated (by ```autoscale_enabled``` setting above), when it's inactive.    
But if you want to remove, please run the following. (After the command completed, you must remove Batch AI resource in Azure Portal now...)

In [11]:
# Delete cluster (nbodes) and remove from AML workspace
batchai_target = BatchAiCompute(workspace=ws, name='mycluster01')
batchai_target.delete()
# Then please remove Batch AI resource manually (in Azure Portal)